#### ST449 Final Project

#### Connect4 Best Bots and Modifications

In [10]:
# imports
import numpy as np
import sys
import pygame
import math
import random
import pandas as pd
# from aima_python_master.utils4e  import *
# from aima_python_master.games4e  import *


#### Generating the Board

In [11]:
ROW_COUNT = 6
COLUMN_COUNT = 7

def create_board(ROW_COUNT, COLUMN_COUNT):
	board = np.zeros((ROW_COUNT,COLUMN_COUNT), dtype= int)
	return board

def drop_piece(board, row, col, piece):
	board[row][col] = piece

def is_valid_location(board, col):
	return board[ROW_COUNT-1][col] == 0

def get_next_open_row(board, col):
	for r in range(ROW_COUNT):
		if board[r][col] == 0:
			return r

def print_board(board):
	print(np.flip(board, 0))

def winning_move(board, piece):
	# Check horizontal locations for win
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT):
			if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
				return True

	# Check vertical locations for win
	for c in range(COLUMN_COUNT):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
				return True

	# Check positively sloped diagonals
	for c in range(COLUMN_COUNT-3):
		for r in range(ROW_COUNT-3):
			if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
				return True

	# Check negatively sloped diagonals
	for c in range(COLUMN_COUNT-3):
		for r in range(3, ROW_COUNT):
			if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
				return True


def generate_board_size(rows, columns):
	board = create_board(rows, columns)
	return board 

#### Players (Bots)

In [5]:
# Random bot
def bot_move(board, piece):
    valid_locations = [col for col in range(COLUMN_COUNT) if is_valid_location(board, col)]
    col = random.choice(valid_locations)
    row = get_next_open_row(board, col)
    drop_piece(board, row, col, piece)

In [14]:
class Connect4Game:
    def __init__(self, board, piece):
        self.board = board
        self.piece = piece

    def to_move(self, state):
        return self.piece

    def actions(self, state):
        return [col for col in range(COLUMN_COUNT) if is_valid_location(state, col)]

    def result(self, state, action):
        new_state = state.copy()
        row = get_next_open_row(new_state, action)
        drop_piece(new_state, row, action, self.piece)
        return new_state

    def terminal_test(self, state):
        return winning_move(state, self.piece) or winning_move(state, 3 - self.piece) or len(self.actions(state)) == 0

    def utility(self, state, player): # Should go over this
        if winning_move(state, player):
            return 1
        elif winning_move(state, 3 - player):
            return -1
        else:
            return 0
        


In [15]:
# Alpha Beta (Yonatan)
def alpha_beta_search(state, game):
    """Search game to determine best action; use alpha-beta pruning.
    As in [Figure 5.7], this version searches all the way to the leaves."""

    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta):
        if game.terminal_test(state):
            return game.utility(state, player)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_search:
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action


def alpha_beta_bot(board, piece):
    game = Connect4Game(board, piece)
    col = alpha_beta_search(board, game)
    row = get_next_open_row(board, col)
    drop_piece(board, row, col, piece)

#### Play Game

In [ ]:
board = generate_board_size(6,7)
game_over = False
turn = 0

while not game_over:
    # Bot 1 move
    if turn == 0:
        bot_move(board, 1)
        if winning_move(board, 1):
            print("Bot 1 wins!")
            game_over = True

    # Bot 2 move
    else:
        alpha_beta_bot(board, 2)
        if winning_move(board, 2):
            print("Bot 2 wins!")
            game_over = True

    print_board(board)
    turn += 1
    turn = turn % 2
    

In [ ]:
## Output shpould be matrix of how well each bot did in a number of n games against the other bots (first move randomly chosen)

#### Play Game (Different Rules)